# Single process State Machine

In [3]:
%%bash

cat <<EOF > sm1/heartbeat.vhdl
LIBRARY IEEE;
USE IEEE.STD_LOGIC_1164.ALL;

ENTITY heartbeat IS
  PORT (clk : OUT STD_LOGIC);
END heartbeat;

ARCHITECTURE behaviour OF heartbeat IS
  CONSTANT clk_period : TIME := 10 ns;
BEGIN
  -- Clock process definition
  clk_process : PROCESS
  BEGIN
    clk <= '0';
    WAIT FOR clk_period/2;
    clk <= '1';
    WAIT FOR clk_period/2;
  END PROCESS;
END behaviour;
EOF

In [4]:
%%bash

cat <<EOF > sm1/sm1.vhdl
library IEEE;
use IEEE.STD_LOGIC_1164.ALL;

-- Uncomment the following library declaration if using
-- arithmetic functions with Signed or Unsigned values
--use IEEE.NUMERIC_STD.ALL;

-- Uncomment the following library declaration if instantiating
-- any Xilinx leaf cells in this code.
--library UNISIM;
--use UNISIM.VComponents.all;

entity sm1 is
    port(
      Clk : in std_logic;
      Rst : in std_logic;
      Sel : in std_logic_vector(3 downto 0);
      Dout   : out std_logic_vector(3 downto 0)  
    );  
end sm1;

architecture Behavioral of sm1 is

type state_t is (S0, S1, S2, S3);
signal state : state_t;
signal NextState : state_t;
constant Value0 : std_logic_vector(3 downto 0) := x"A";
constant Value1 : std_logic_vector(3 downto 0) := x"B";
constant Value2 : std_logic_vector(3 downto 0) := x"C";
constant Value3 : std_logic_vector(3 downto 0) := x"D";

begin

-- ONE process sm1

process(Clk) is
begin
   if rising_edge(Clk) then
       if Rst = '1' then
           State <= S0;
           Dout  <= Value0;
       else
         case State is
            when S0 =>   if Sel(1) = '1' then
                              State <= S1;
                              Dout <= Value1;
                           else
                              State <= S2;
                           end if;

            when S1 =>   if Sel(2) = '1' then
                              State <= S2;
                              Dout <= Value2;
                           else
                              State <= S3;
                           end if;
            when S2 =>   if Sel(3) = '1' then
                              State <= S3;
                              Dout <= Value3;
                           else
                              State <= S0;
                           end if;
            when S3 =>   if Sel(0) = '1' then
                              State <= S0;
                              Dout <= Value0;
                           else
                              State <= S1;
                           end if;

            when others =>  Dout <= Value0;
                                       State <= S0;
         end case;
      end if;
   end if;
end process;

-- END process sm1

end Behavioral;

EOF

In [5]:
%%bash

cat <<EOF > sm1/sm1_tb.vhdl
LIBRARY IEEE;
USE IEEE.STD_LOGIC_1164.ALL;


ENTITY sm1_tb IS
END sm1_tb;

ARCHITECTURE behaviour OF sm1_tb IS
  --  Declaration of the component that will be instantiated.
  COMPONENT sm1
    PORT (Clk : IN STD_LOGIC;
          Rst : IN STD_LOGIC;
          Sel : IN STD_LOGIC_VECTOR(3 DOWNTO 0);
          Dout : OUT STD_LOGIC_VECTOR(3 DOWNTO 0));
  END COMPONENT;

  COMPONENT heartbeat
    PORT (clk : OUT STD_LOGIC);
  END COMPONENT;

  --  Specifies which entity is bound with the component.
  FOR sm1_0 : sm1 USE ENTITY work.sm1;
  FOR heartbeat_0 : heartbeat USE ENTITY work.heartbeat;

  SIGNAL clk : STD_LOGIC;
  SIGNAL Rst : STD_LOGIC := '0';
  SIGNAL Sel : STD_LOGIC_VECTOR(3 DOWNTO 0) := "0000"; 
  SIGNAL Dout : STD_LOGIC_VECTOR(3 DOWNTO 0);

BEGIN
  --  Component instantiation.
  sm1_0 : sm1 PORT MAP(clk => clk, Rst => Rst, Sel => Sel, Dout => Dout);
  heartbeat_0 : heartbeat PORT MAP(clk => clk);

  --  This process does the real job.
  PROCESS
    BEGIN
    wait for 100 ns;
      Sel <= "1111";  -- Select state S1
    WAIT for 1000 ns;
      Sel <= "1100";  -- Select state S2
    WAIT for 1000 ns;
      Sel <= "1000";  -- Select state S3
  Wait;
  END PROCESS;

END ARCHITECTURE behaviour;
EOF

In [9]:
%%bash
cd sm1
rm -f work* *.ghw
ghdl -a heartbeat.vhdl
ghdl -a sm1.vhdl
ghdl -a sm1_tb.vhdl
ghdl -r sm1_tb --stop-time=5000ns --wave=sm1_tb.ghw

ghdl:info: simulation stopped by --stop-time @5us


Clean up cell

In [10]:
%%bash
cd sm1
rm -f work* *.ghw